In [6]:
!pip install ipynb

In [7]:
!pip install pandas_profiling

     |████████████████████████████████| 153kB 2.1MB/s eta 0:00:01
     |████████████████████████████████| 614kB 7.2MB/s eta 0:00:01


     |████████████████████████████████| 737kB 1.6MB/s eta 0:00:01
  Created wheel for pandas-profiling: filename=pandas_profiling-2.4.0-py2.py3-none-any.whl size=195587 sha256=b65da464fea83dc78ebd78ab610d35f313cd4bb63ba1f4e0dc92ca2436e079c8
  Stored in directory: /home/guillermo/snap/code/common/.cache/pip/wheels/b2/4b/40/ff4633a59f4e7fbdfec19b8e10e353b54b5eef125c31afe51a
  Created wheel for confuse: filename=confuse-1.0.0-cp37-none-any.whl size=17486 sha256=3f04d8c902f480b4cbfb6f77945a7f9a9d8c0b71930ca502f5d9c2eb52c0bc42
  Stored in directory: /home/guillermo/snap/code/common/.cache/pip/wheels/b0/b2/96/2074eee7dbf7b7df69d004c9b6ac4e32dad04fb7666cf943bd
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp37-none-any.whl size=27086 sha256=96c9e977377b9ac277e63f2c3c586cc38e888b8d812312da2f4d9ab59d389d3c
  Stored in directory: /home/guillermo/snap/code/common/.cache/pip/wheels/43/07/ac/7c5a9d708d65247ac1f94066cf1db075540b85716c30255459
Successfully built pandas-profiling confuse htmlmi

In [8]:
from ipynb.fs.full.tokyo_NLP import *

FileNotFoundError: [Errno 2] No such file or directory: 'listings_tokyo.csv.gz'

In [5]:
df = pd.get_dummies(listings_clean_model)

NameError: name 'pd' is not defined

In [ ]:
df.shape

In [ ]:
# Create correlation matrix
corr_matrix = df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

In [ ]:
to_drop 

In [ ]:
numerical_col = ['accommodates','bathrooms',
                 'cleaning_fee',
                'minimum_nights','price']

In [ ]:
df[numerical_col].hist(figsize=(10,11));

In [ ]:
# Log transforming columns because some features are highly skewed. 
numerical_col = [i for i in numerical_col if i not in ['accommodates']] # Removing items not to be transformed

for col in numerical_col:
        df[col] = df[col].astype('float64').replace(0.0, 0.01) # Replacing 0s with 0.01
        df[col] = np.log(df[col])

In [ ]:
#Target
y_price = df['price']
X_price = df.drop('price', axis=1)

In [ ]:
# Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_price = pd.DataFrame(scaler.fit_transform(X_price), columns=list(X_price.columns))

In [ ]:
from sklearn.model_selection import train_test_split
X_train_price, X_test_price, y_train_price, y_test_price = train_test_split(
                                                           X_price, y_price, test_size=0.2, random_state=42)

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error,r2_score,explained_variance_score

xgb_reg = xgb.XGBRegressor()
xgb_reg.fit(X_train_price, y_train_price)
training_preds_xgb_reg = xgb_reg.predict(X_train_price)
val_preds_xgb_reg = xgb_reg.predict(X_test_price)

print("\nTraining MSE:", round(mean_squared_error(y_train_price, training_preds_xgb_reg),3))
print("Validation MSE:", round(mean_squared_error(y_test_price, val_preds_xgb_reg),3))

In [ ]:
print("\nTraining r2:", round(r2_score(y_train_price, training_preds_xgb_reg),3))
print("Validation r2:", round(r2_score(y_test_price, val_preds_xgb_reg),3))

In [ ]:
ft_weights_xgb_reg = pd.DataFrame(xgb_reg.feature_importances_, columns=['weight'], index=X_train_price.columns)
ft_weights_xgb_reg.sort_values('weight', inplace=True)
#ft_weights_xgb_reg

In [ ]:
# Drop feature weights of 0 
ft_weights_xgb_reg = ft_weights_xgb_reg[(ft_weights_xgb_reg != 0).all(1)]

In [ ]:
# Plotting feature importances
import matplotlib.pyplot as plt
plt.figure(figsize=(8,32))
plt.barh(ft_weights_xgb_reg.index, ft_weights_xgb_reg.weight, align='center') 
plt.title("Feature importances in the XGBoost model", fontsize=14)
plt.xlabel("Feature importance")
plt.margins(y=0.01)
plt.show()

### Lets use availability_30 as Target

In [ ]:
# #Target
# y_30 = df['availability_30']
# X_30 = df.drop(['availability_30','availability_90'],axis=1)

In [ ]:
# X_30 = pd.DataFrame(scaler.fit_transform(X_30), columns=list(X_30.columns))
# X_train_30, X_test_30, y_train_30, y_test_30 = train_test_split(X_30, y_30, test_size=0.2, random_state=42)

In [ ]:
# xgb_reg = xgb.XGBRegressor()
# xgb_reg.fit(X_train_30, y_train_30)
# training_preds_xgb_reg = xgb_reg.predict(X_train_30)
# val_preds_xgb_reg = xgb_reg.predict(X_test_30)

# print("\nTraining MSE:", round(mean_squared_error(y_train_30, training_preds_xgb_reg),3))
# print("Validation MSE:", round(mean_squared_error(y_test_30, val_preds_xgb_reg),3))

In [ ]:
# ft_weights_xgb_reg = pd.DataFrame(xgb_reg.feature_importances_, columns=['weight'], index=X_train_30.columns)
# ft_weights_xgb_reg.sort_values('weight', inplace=True)

In [ ]:
# # Drop feature weights of 0 
# ft_weights_xgb_reg = ft_weights_xgb_reg[(ft_weights_xgb_reg != 0).all(1)]

In [ ]:
# # Plotting feature importances
# import matplotlib.pyplot as plt
# plt.figure(figsize=(8,30))
# plt.barh(ft_weights_xgb_reg.index, ft_weights_xgb_reg.weight, align='center') 
# plt.title("Feature importances in the XGBoost model", fontsize=14)
# plt.xlabel("Feature importance")
# plt.margins(y=0.01)
# plt.show()

### Availability_30 has a bigger MSE. 

### Lets use availability_90 as the Target

In [ ]:
# #Target
# y_90 = df['availability_90']
# X_90 = df.drop(['availability_30','availability_90'],axis=1)

# X_90 = pd.DataFrame(scaler.fit_transform(X_90), columns=list(X_90.columns))
# X_train_90, X_test_90, y_train_90, y_test_90 = train_test_split(X_90, y_90, test_size=0.2, random_state=42)

# xgb_reg = xgb.XGBRegressor()
# xgb_reg.fit(X_train_90, y_train_90)
# training_preds_xgb_reg = xgb_reg.predict(X_train_90)
# val_preds_xgb_reg = xgb_reg.predict(X_test_90)

# print("\nTraining MSE:", round(mean_squared_error(y_train_90, training_preds_xgb_reg),3))
# print("Validation MSE:", round(mean_squared_error(y_test_90, val_preds_xgb_reg),3))

# ft_weights_xgb_reg = pd.DataFrame(xgb_reg.feature_importances_, columns=['weight'], index=X_train_90.columns)
# ft_weights_xgb_reg.sort_values('weight', inplace=True)
# # Drop feature weights of 0 
# ft_weights_xgb_reg = ft_weights_xgb_reg[(ft_weights_xgb_reg != 0).all(1)]

# # Plotting feature importances
# import matplotlib.pyplot as plt
# plt.figure(figsize=(8,30))
# plt.barh(ft_weights_xgb_reg.index, ft_weights_xgb_reg.weight, align='center') 
# plt.title("Feature importances in the XGBoost model", fontsize=14)
# plt.xlabel("Feature importance")
# plt.margins(y=0.01)
# plt.show()

### Availability_90 has a huge MSE. 

## Neural Net

In [ ]:
from keras import models, layers, optimizers, regularizers

### Build Model without regularizers

In [ ]:
X_train_price, X_test_price, y_train_price, y_test_price = train_test_split(
                                                           X_price, y_price, test_size=0.2, random_state=42)

In [ ]:
# Building the model
nn = models.Sequential()
nn.add(layers.Dense(128, input_shape=(X_train_price.shape[1],), activation='relu'))
nn.add(layers.Dense(256, activation='relu'))
nn.add(layers.Dense(256, activation='relu'))
nn.add(layers.Dense(512, activation='relu'))
nn.add(layers.Dense(1, activation='linear'))

# Compiling the model
nn.compile(loss='mean_squared_error',
            optimizer='adam',
            metrics=['mean_squared_error'])

# Model summary
print(nn.summary())

In [ ]:
# Training the model
nn_history = nn.fit(X_train_price,
                  y_train_price,
                  epochs=150,
                  batch_size=256,
                  validation_split = 0.1)

In [ ]:
def nn_model_evaluation(model,X_train, X_test, y_train, y_test, skip_epochs=0,):
    """
    For a given neural network model that has already been fit, prints for the train and tests sets the MSE and r squared
    values, a line graph of the loss in each epoch, and a scatterplot of predicted vs. actual values with a line
    representing where predicted = actual values. Optionally, a value for skip_epoch can be provided, which skips that
    number of epochs in the line graph of losses (useful in cases where the loss in the first epoch is orders of magnitude
    larger than subsequent epochs). Training and test sets can also optionally be specified.
    """

    # MSE and r squared values
    y_test_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)
    print("Training MSE:", round(mean_squared_error(y_train, y_train_pred),4))
    print("Validation MSE:", round(mean_squared_error(y_test, y_test_pred),4))
    print("\nTraining r2:", round(r2_score(y_train, y_train_pred),4))
    print("Validation r2:", round(r2_score(y_test, y_test_pred),4))
    
    # Line graph of losses
    model_results = model.history.history
    plt.plot(list(range((skip_epochs+1),len(model_results['loss'])+1)), model_results['loss'][skip_epochs:], label='Train')
    plt.plot(list(range((skip_epochs+1),len(model_results['val_loss'])+1)), model_results['val_loss'][skip_epochs:], label='Test', color='green')
    plt.legend()
    plt.title('Training and test loss at each epoch', fontsize=14)
    plt.show()

In [ ]:
nn_model_evaluation(nn,X_train_price,X_test_price, y_train_price, y_test_price, skip_epochs=10)

### Build Model with regularizers

In [ ]:
# Building the model
nn2 = models.Sequential()
nn2.add(layers.Dense(128, input_shape=(X_train_price.shape[1],), kernel_regularizer=regularizers.l1(0.005), activation='relu'))
nn2.add(layers.Dense(256, kernel_regularizer=regularizers.l1(0.005), activation='relu'))
nn2.add(layers.Dense(256, kernel_regularizer=regularizers.l1(0.005), activation='relu'))
nn2.add(layers.Dense(512, kernel_regularizer=regularizers.l1(0.005), activation='relu'))
nn2.add(layers.Dense(1, activation='linear'))

# Compiling the model
nn2.compile(loss='mean_squared_error',
            optimizer='adam',
            metrics=['mean_squared_error'])

# Model summary
print(nn2.summary())

In [ ]:
# Training the model
nn2_history = nn2.fit(X_train_price,
                  y_train_price,
                  epochs=150,
                  batch_size=256,
                  validation_split = 0.1)

In [ ]:
nn_model_evaluation(nn2,X_train_price,X_test_price, y_train_price, y_test_price, skip_epochs=10)

### Neural Net with Kernal Regularizers is best performing with lowest MSE and higest R2 score. 

In [ ]:
import pickle 

# Pickle the darn thing! 🥒
model_name = '/Users/karthikmahendra/Desktop/AirBnB/ABB.pkl' # path to where you want the file
pickle.dump(xgb_reg, open(model_name, 'wb')) # kitty = the name of your model 

In [ ]:
X_tst = X_train_price.iloc[0]
y_tst = y_train_price.iloc[0]

In [ ]:
loaded_model = pickle.load(open(model_name, 'rb'))
result = loaded_model.score(X_tst, y_tst)
print(result)

In [ ]:
saved_model